# Import

In [3]:
# import yfinance as yf
import pandas as pd
import os
from os.path import join
from datetime import datetime, timezone
import pandas as pd
from yforseer.scraping import update_raw_tables, load_ticker_list, RawDF_Schema
data_dir = 'data/yahoo/tickerdaily'

# Scraping

In [ ]:

ticker_list = load_ticker_list('yforseer/ticker_list.csv')
update_raw_tables(data_dir, ticker_list)

# Combine dataframe

In [1]:
from yforseer.prepare_data import combine_df
combine_df(raw_data_dir = 'data/yahoo/tickerdaily', 
           save_df_pth = 'data/yahoo/artifacts/combined.csv')



# Select time period, interpolate, stack arrays

In [6]:
from yforseer.prepare_data import CombinedDF_Schema

In [ ]:
combined_df = pd.read_csv('data/yahoo/artifacts/combined.csv').astype(CombinedDF_Schema)

In [9]:
combined_df

,ticker_id,ticker_code,Date,Close
0,0,1COV.DE,1996-10-30 00:00:00+01:00,10.742868
1,0,1COV.DE,1996-10-31 00:00:00+01:00,10.886429
2,0,1COV.DE,1996-11-01 00:00:00+01:00,10.937512
3,0,1COV.DE,1996-11-04 00:00:00+01:00,11.022678
4,0,1COV.DE,1996-11-05 00:00:00+01:00,11.229510
...,...,...,...,...
66551,10,VOW3.DE,2024-07-10 00:00:00+02:00,106.250000
66552,10,VOW3.DE,2024-07-11 00:00:00+02:00,107.150002
66553,10,VOW3.DE,2024-07-12 00:00:00+02:00,108.550003
66554,10,VOW3.DE,2024-07-15 00:00:00+02:00,108.199997
